In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import os
import json
import numpy as np
import time
import random
import json
from concurrent.futures import ThreadPoolExecutor, as_completed

In [2]:
from glob import glob

files = glob('url/*.json')
urls = []
for f in files:
    with open(f) as fopen:
        urls.extend(json.load(fopen))
urls = sorted(list(set(urls)))
len(urls)

12934

In [3]:
# !rm -rf page
!mkdir page

In [4]:
r = requests.get(urls[0])
soup = BeautifulSoup(r.content, "lxml")

In [5]:
soup.find('h1').text

'《Clash of Clans》Tahun Arnab TVC Promosi Animasi Mendedah, Kulit yang Terhad Masa dan Adegan Menampil dengan Kejutan!'

In [6]:
soup.find('div', {'class': 'content-inner'}).text

'\nMenjelang Tahun Baru Arnab, 《Clash of Clans》dilancarkan oleh pengilang terkenal Supercell Finland, sampai sampai untuk mengucap selamat tinggal kepada yang lama dan menyambut yang baru, banyak kandungan terhad tahun baru cina dan kebajikan terhad dilancarkan.《Clash of Clans》yang sentiasa mementingkan pasaran Asia, kali ini telah membawa banyak kandungan yang mengutamakan zodiak cina “arnab” , bersama-sama menyambut Tahun Baru Cina Arnab dengan para pemain!\n\n\n\n\n\n\nDua tahun yang lepas, 《Clash of Clans》melancarkan video promosi yang bertema “Pasar Tahun Baru Cina” dan “Tembok Besar” berturut-turut masa tempoh Tahun Baru Cina, bergabung dengan kandungan untuk tahun baru dalam permainan, telah meninggalkan imej yang mendalam kepada semua orang. Tahun Baru Cina Arnab, 《Clash of Clans》 sekali lagi membawa promosi video animasi yang menarik khas untuk tahun baru cina!\n\n\n\nDengan tema “Semoga Arnab Bersamamu” (May the rabbit with you) sebagai video promosi, membenarkan ahli sihir d

In [9]:
def crawl(url, filename):
    if os.path.exists(filename):
        return
    
    while True:
        try:
            r = requests.get(url)
            break
        except:
            time.sleep(1.0)
    soup = BeautifulSoup(r.content, "lxml")
    
    try:
        title = soup.find('h1').text
    except Exception as e:
        title = None

    try:
        body = soup.find('div', {'class': 'content-inner'}).text
    except Exception as e:
        body = None

    data = {
        'url': url,
        'title': title,
        'body': body,
    }
    with open(filename, 'w') as fopen:
        json.dump(data, fopen)

In [10]:
max_worker = 10

for i in tqdm(range(0, len(urls), max_worker)):
    a = np.arange(i, i + max_worker)
    urls_ = [(urls[k], os.path.join('page', f'{k}.json')) for k in a]
    
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(crawl, url[0], url[1]): url for url in urls_}

        for future in as_completed(futures):
            future.result()

100%|████████████████████████████████████▉| 1293/1294 [5:53:48<00:16, 16.42s/it]


IndexError: list index out of range